# **_Bank GoodCredit wants to predict credit score for current credit card customers._**

#  Dataset Overview – Bank GoodCredit Credit Risk Project

## Business Case:

Bank GoodCredit wants to assess the **creditworthiness** of its credit card customers by predicting if a customer is likely to **default** (i.e., become 30+ days past due).
This helps reduce credit risk and improve loan decisions.

**Target Variable:**
`Bad_label`

* `0` → Good credit history
* `1` → Bad credit history (default risk)

---

###  **Dataset Description**

The data is stored in a **MySQL database** with the following three main tables:

---

#### 1. **Cust\_Account**

Customer’s historical account and payment information.

* `customer_no`
* `opened_dt`, `last_paymt_dt`, `closed_dt`
* `cur_balance_amt`, `creditlimit`, `cashlimit`
* `paymenthistory1`, `paymentfrequency`
* `rateofinterest`, `actualpaymentamount`
* ... *(more account-specific fields)*

---

#### 2. **Cust\_Enquiry**

Contains credit enquiry history for each customer.

* `customer_no`
* `enquiry_dt`, `enq_purpose`, `enq_amt`
* Used to evaluate customer's recent credit-seeking behavior.

---

#### 3. **Cust\_Demographics**

Includes demographic and anonymized model features.

* `customer_no`
* `feature_1` to `feature_79`
* `Bad_label` (target variable)

---

### **Project Goal**

Build a classification model using this data to:

* Analyze customer behavior
* Select relevant features
* Predict default risk (Bad\_label)
* Evaluate performance using Gini score and decile ranking

In [ ]:
!pip install mysql-connector-python

In [ ]:
import os  # Used to interact with the operating system
import mysql.connector # Used to connect to MySQL database and load datasets
import pandas as pd # Used for data cleaning, manipulation, and analysis
import numpy as np # Used for numerical operations and handling arrays
import matplotlib.pyplot as plt # Used for data visualization (creating charts and plots)
import warnings # Used to manage warning messages
import seaborn as sns # Used for statistical data visualization

# Ignore warning messages to keep the output clean
warnings.filterwarnings("ignore")

In [ ]:
## Making connection to the database server
connection = mysql.connector.connect(
    host="18.136.157.135",  # Database server IP address
    user="dm_team1",  # Username for authentication
    password="DM!$Team&279@20!"  # Password for authentication
)

In [ ]:
# Create a cursor object to interact with the MySQL database
cursor = connection.cursor()

# Execute the SQL command to list all available databases
cursor.execute('SHOW DATABASEs')

# Print each database name retrieved from the server
for db in cursor:
    print(db)

In [ ]:
# Establish connection to the MySQL database server
connection = mysql.connector.connect(
    host="18.136.157.135",  # Database server IP address
    port = "3306",
    user="dm_team1",  # Username for authentication
    password="DM!$Team&279@20!",  # Password for authentication
    database="project_banking"  # Specify the database to connect to
)

#### Retrieve all tables from the 'project_banking' database
db_Tables = pd.read_sql_query("SHOW TABLES", connection)

# Display the list of tables
db_Tables

In [ ]:
# Fetch all records from the Cust_Account table and store them in a DataFrame
customer_acc = pd.read_sql_query('SELECT * FROM Cust_Account',connection)

# Display the retrieved data
customer_acc

In [ ]:
# Fetch all records from the Cust_Demographics table and store them in a DataFrame
customer_demo = pd.read_sql_query('SELECT * FROM Cust_Demographics',connection)

# Display the retrieved data
customer_demo

In [ ]:
# Fetch all records from the Cust_Enquiry table and store them in a DataFrame
customer_enq = pd.read_sql_query('SELECT * FROM Cust_Enquiry',connection)

# Display the retrieved data
customer_enq

In [ ]:
# Export the 'customer_acc' DataFrame to a CSV file named 'customer_acc.csv'
# Set index=False to avoid writing row numbers (index) into the file
customer_acc.to_csv('customer_acc.csv', index=False)

# Export the 'customer_demo' DataFrame to a CSV file named 'customer_demo.csv'
# This typically contains demographic information like age, gender, etc.
customer_demo.to_csv('customer_demo.csv', index=False)

# Export the 'customer_enq' DataFrame to a CSV file named 'customer_enq.csv'
# This usually includes customer enquiries or interaction history
customer_enq.to_csv('customer_enq.csv', index=False)

In [ ]:
df1 = pd.read_csv('customer_acc.csv')
df1

In [ ]:
df2 = pd.read_csv('customer_demo.csv')
df2

In [ ]:
df3 = pd.read_csv('customer_enq.csv')
df3

In [ ]:
# Display the number of rows and columns in the  DataFrame
# Output will be in the format: (rows, columns)
df1.shape

In [ ]:
# Display the number of rows and columns in the  DataFrame
# Output will be in the format: (rows, columns)
df2.shape

In [ ]:
# Display the number of rows and columns in the  DataFrame
# Output will be in the format: (rows, columns)
df3.shape

In [ ]:
# Remove duplicate rows in df1 based on the 'customer_no' column
# Keeps the first occurrence and drops any additional entries with the same 'customer_no'
df1 = df1.drop_duplicates(subset='customer_no')

# Remove duplicate rows in df2 based on the 'customer_no' column
# Ensures each customer appears only once in the demographic data
df2 = df2.drop_duplicates(subset='customer_no')

# Remove duplicate rows in df3 based on the 'customer_no' column
# Cleans up the enquiry data by removing repeated entries for the same customer
df3 = df3.drop_duplicates(subset='customer_no')

In [ ]:
# Merge df1 (customer account data) and df3 (customer enquiry data) using an inner join on 'customer_no'
# This keeps only the rows where 'customer_no' exists in both df1 and df3
 # Then merge the result with df2 (customer demographic data), again using an inner join on 'customer_no'
          # This ensures the final DataFrame 'data' contains only those customers present in all three DataFrames
data = df1.merge(df3, how="inner", on="customer_no").merge(df2, how="inner", on="customer_no")

In [ ]:
# Display merged DataFrame
data

## Basic Check :-

In [ ]:
data.head()  #showing first 5 rows of dataset

In [ ]:
data.tail()  # showung last 5 rows of dataset

In [ ]:
data.describe(include="O") # showing categorical column's information

In [ ]:
data.info()#showing information about all columns of data set

In [ ]:
data.shape

In [ ]:
data.isnull().sum()

## Basic Data Preprocessing :-

In [ ]:
date_col = [
    'dt_opened_x', 'upload_dt_x', 'last_paymt_dt',
    'reporting_dt', 'paymt_str_dt', 'paymt_end_dt', 'dt_opened_y', 'entry_time',
    'feature_2', 'feature_21', 'feature_53', 'feature_54','dt_opened', 
    'upload_dt_y', 'enquiry_dt','feature_30','feature_39','feature_63','feature_75']


cat_col = [
    'paymenthistory1', 'paymenthistory2', 'feature_1', 'feature_5', 'feature_8',
    'feature_9', 'feature_11', 'feature_12', 'feature_13', 'feature_15', 'feature_16',
    'feature_18', 'feature_20', 'feature_22', 'feature_23', 'feature_24', 'feature_27',
    'feature_28', 'feature_32', 'feature_33', 'feature_36', 'feature_37', 'feature_38',
    'feature_43', 'feature_45', 'feature_46', 'feature_47', 'feature_48', 'feature_50',
    'feature_77', 'feature_51', 'feature_57', 'feature_58', 'feature_59', 'feature_60',
    'feature_61','feature_62', 'feature_72', 'feature_73', 'feature_79'
]


num_col = [
    'customer_no', 'acct_type', 'owner_indic', 'high_credit_amt', 'cur_balance_amt',
    'amt_past_due', 'creditlimit', 'cashlimit', 'rateofinterest', 'paymentfrequency',
    'actualpaymentamount', 'feature_3', 'feature_4', 'feature_6', 'feature_7',
    'feature_10', 'feature_14', 'feature_17', 'feature_19', 'feature_25', 'feature_26',
    'feature_29', 'feature_35', 'feature_40', 'feature_41', 'feature_42',
    'feature_44', 'feature_52', 'feature_55', 'feature_56', 'feature_64', 'feature_65',
    'feature_66', 'feature_67', 'feature_68', 'feature_69', 'feature_71', 'feature_74',
    'Bad_label', 'feature_34', 'feature_31', 'enq_purpose', 'enq_amt',
    'feature_49', 'feature_76', 'feature_78'
]

In [ ]:
pd.set_option('display.max_columns',None)
pd.set_option('display.max_rows',None)

In [ ]:
# Loop through each column name in the list 'cat_col'
# These are assumed to be categorical columns in the DataFrame
for i in cat_col:
    
    # Convert the data type of each column to 'object' (which is used for strings/categorical data in pandas)
    data[i] = data[i].astype("object")

In [ ]:
# Loop through each column name in the list 'num_col'
# These are assumed to be numerical columns in the DataFrame
for i in num_col:
    
    # Convert each column to numeric type using pandas 'pd.to_numeric'
    # The 'errors='coerce'' argument replaces any non-numeric values with NaN (Not a Number)
    data[i] = pd.to_numeric(data[i], errors='coerce')

In [ ]:
# Convert the 'opened_dt' column to a datetime format using 'datetime64[ns]'
# This ensures that the 'opened_dt' column is treated as a datetime type in pandas
data["opened_dt"] = data["opened_dt"].astype("datetime64[ns]")

# Convert the 'closed_dt' column to a datetime format using 'datetime64[ns]'
# This ensures that the 'closed_dt' column is treated as a datetime type in pandas
data["closed_dt"] = data["closed_dt"].astype("datetime64[ns]")

# Feature Engineering :-

> * feature engineering is the process of creating new features or transforming existing ones to improve model performance. Below are engineered features for credit risk analysis based on account history, enquiry records, and demographic data.


##  Data Preprocessing Overview

Data preprocessing prepares raw data for modeling by cleaning inaccuracies, handling missing values, and encoding categorical features. It also involves scaling numerical data and selecting relevant features. This process ensures data quality, enhances model performance, and improves training efficiency.

---

### **Data Cleaning**

Removing or fixing inaccuracies in the dataset, such as:

* Duplicate records
* Inconsistent formatting
* Irrelevant or redundant columns

---

### **Handling Missing Values**

Missing data can negatively impact models. Common strategies include:

* Filling with statistical values like **mean**, **median**, or **mode**
* Using placeholders (e.g., `"Unknown"` or `-999`)
* Dropping rows or columns with excessive missing values

---

### **Encoding Categorical Variables**

Machine learning algorithms require numerical input. Categorical data like `"Gender"` or `"City"` must be converted using:

* **Label Encoding** – for ordinal categories
* **One-Hot Encoding** – for nominal categories

---

### **Scaling and Normalization**

To ensure that all features contribute equally to the model:

* **Normalization** scales values between 0 and 1
  
---

### **Feature Selection and Extraction**

Improving model performance by:

* **Feature Selection**: Keeping only the most relevant features based on statistical tests or model-based importance
* **Feature Extraction**: Creating new features (e.g., combining year and month into a time series index) that reveal hidden patterns or reduce dimensionality.

In [ ]:
# Check null values only in specified categorical columns
cat_null = data[cat_col].isnull().sum()

# Display only columns that actually have null values
cat_null[cat_null > 0]

In [ ]:
# Check null values only in specified categorical columns
num_null = data[num_col].isnull().sum()

# Display columns
num_null

In [ ]:
for col in cat_null.index:
    mode_value = data[col].mode()[0]  # Get the mode (most frequent value)
    data[col].fillna(mode_value, inplace=True)

In [ ]:
# Check null values only in specified categorical columns
cat_null = data[cat_col].isnull().sum()

# Display only columns that actually have null values
cat_null

In [ ]:
for col in num_null.index:
    median_value = data[col].median()  # Get median of the column
    data[col].fillna(median_value, inplace=True)

In [ ]:
# Check null values only in specified categorical columns
cat_null = data[cat_col].isnull().sum()

# Display only columns that actually have null values
cat_null

In [ ]:
data.shape

In [ ]:
# Drop the columns listed in 'date_col' from the DataFrame 'data'
# 'axis=1' specifies that columns (not rows) should be dropped
# 'inplace=True' modifies the 'data' DataFrame directly without needing to assign the result to a new variable
data.drop(date_col, axis=1, inplace=True)

In [ ]:
# Calculate the mode (most frequent value) of the 'closed_dt' column
# The mode is useful for understanding the most common date in the 'closed_dt' column
# It returns a pandas Series, so you can access the first mode value using [0] if needed
data['closed_dt'].mode()

In [ ]:
# Calculate the mode (most frequent value) of the 'opened_dt' column
# The mode is useful for understanding the most common date in the 'opened_dt' column
# It returns a pandas Series, so you can access the first mode value using [0] if needed
data['opened_dt'].mode()

In [ ]:
# Replace missing (null) values in the 'closed_dt' column with a specific date ('2015-02-04')
# 'data['closed_dt'].isnull()' identifies rows where 'closed_dt' is null
# 'data.loc' is used to update the values in these rows, setting them to the specified date
data.loc[data['closed_dt'].isnull(), 'closed_dt'] = '2015-02-04'


In [ ]:
# Replace missing (null) values in the 'opened_dt' column with a specific date ('2015-03-31')
# 'data['opened_dt'].isnull()' identifies rows where 'opened_dt' is null
# 'data.loc' is used to update the values in these rows, setting them to the specified date
data.loc[data['opened_dt'].isnull(),'opened_dt'] = '2015-03-31'

In [ ]:
data.isnull().sum()

In [ ]:
# Drop specific columns from the 'data' DataFrame
# The list inside the 'drop()' function contains the column names to be removed
# 'axis=1' specifies that columns (not rows) should be dropped
# 'inplace=True' modifies the original 'data' DataFrame directly without creating a new DataFrame
data.drop(['customer_no', 'paymenthistory1', 'paymenthistory2', 'feature_10', 'feature_18', 
           'feature_20', 'feature_22', 'feature_24', 'feature_45', 'feature_47', 'feature_49', 
           'feature_61', 'feature_77'], axis=1, inplace=True)

In [ ]:
# Select columns from the 'data' DataFrame that have the data type 'object'
# 'object' type typically represents categorical or string data in pandas
# 'select_dtypes(include=['object'])' filters columns with 'object' type
# '.columns' retrieves the column names of the selected columns
# '.tolist()' converts the column names into a list and stores it in 'cat_col'
cat_col = data.select_dtypes(include=['object']).columns.tolist()

In [ ]:
# Select columns from the 'data' DataFrame that have numeric data types: 'float64' or 'int64'
# 'float64' and 'int64' represent continuous and discrete numeric values, respectively
# 'select_dtypes(include=['float64', 'int64'])' filters columns with numeric types
# '.columns' retrieves the column names of the selected columns
# '.tolist()' converts the column names into a list and stores it in 'num_col'
num_col = data.select_dtypes(include=['float64', 'int64']).columns.tolist()

In [ ]:
# Step 1: Ensure 'Bad_label' is numeric
data['Bad_label'] = pd.to_numeric(data['Bad_label'], errors='coerce')

# Step 2: Filter only numeric columns from the DataFrame
num_data = data[num_col].select_dtypes(include=['number'])

# Step 3: Add 'Bad_label' to the numeric data
num_data['Bad_label'] = data['Bad_label']

# Step 4: Compute correlation matrix
corr_matrix = num_data.corr()

# Step 5: Extract correlation of all features with 'Bad_label' (excluding itself)
corr_series = corr_matrix['Bad_label'].drop('Bad_label')

# Step 6: Sort by absolute correlation
corr_sorted = corr_series.abs().sort_values(ascending=False)

In [ ]:
# Loop through each numeric column in the 'num_col' list
for i in num_col:
    
    # If the column is in the 'corr_sorted' list (i.e., has a very weak correlation with 'Bad_label'), skip it
    if i in corr_sorted:
        continue  # Skip the column and move to the next iteration of the loop
    
    # If the column is not in the 'corr_sorted' list, print the column name
    else:
        print(i)  # This will display columns that have a moderate to strong correlation with 'Bad_label'

In [ ]:
data.shape

# EDA :- Exploratory Data Analysis
### Univarinat Analysis

In [ ]:
# Distribution of target variable
data['Bad_label'].value_counts(normalize=True)
data['Bad_label'].value_counts().plot(kind='bar', title='Target Distribution')

In [ ]:
# For numeric
for col in num_col:
    plt.figure()
    sns.histplot(data[col], kde=True)
    plt.title(f"Distribution of {col}")
    plt.show()

# For categorical
for col in cat_col:
    plt.figure()
    data[col].value_counts().plot(kind='bar')
    plt.title(f"Count plot of {col}")
    plt.show()

In [ ]:
# Set the size of the entire figure (25x35) for better spacing between subplots
plt.figure(figsize=(25, 35))  

# Initialize a counter to manage subplot positions
num = 1

# Loop through each categorical column in the 'cat_col' list
for i in cat_col:
    
    # Only plot columns with less than or equal to 50 unique values (low-cardinality)
    if data[i].nunique() <= 50:
        
        # Create a subplot with a grid of 6 rows and 5 columns to fit multiple plots
        plt.subplot(6, 5, num)
        
        # Plot a histogram with a KDE (Kernel Density Estimate) for the categorical feature 'i'
        # 'sns.histplot()' shows the distribution, and 'kde=True' adds the KDE curve
        sns.histplot(data=data, x=i, kde=True, color='skyblue')
        
        # Plot a count plot (bar plot) for the categorical feature 'i'
        # 'sns.countplot()' shows the frequency of each category in the feature
        # 'palette="viridis"' sets the color scheme
        # 'order=data[i].value_counts().index' ensures the categories are ordered by their frequency
        sns.countplot(data=data, x=i, palette='viridis', order=data[i].value_counts().index)
        
        # Set the x-axis label to the column name with a larger font size for better readability
        plt.xlabel(i, fontsize=14)
        
        # Rotate the x-axis labels by 45 degrees and align them to the right for better readability
        plt.xticks(rotation=45, ha='right')
        
        # Adjust the layout of the plots to prevent overlapping and ensure proper spacing between subplots
        plt.tight_layout(pad=2)  
        
        # Increment the subplot number to move to the next subplot position
        num += 1  

# Show the plots after all subplots are created
plt.show()

##  Feature Distribution Summary & Recommendations

###  Well-distributed Features
These features show a good spread across categories and are likely useful for modeling:
- `feature_1`
- `feature_27`
- `feature_32`
- `feature_33`
- `feature_50`

###  Highly Imbalanced Features
These features are heavily skewed toward one or very few values. They may provide limited information and require further evaluation:
- `feature_11`
- `feature_13`
- `feature_23`
- `feature_57`
- `feature_59`
- `feature_60`
- `feature_62`
- `feature_72`
- `feature_73`

###  High Cardinality with Sparse Values
These features have many unique values with low frequency. They may increase model complexity and should be encoded or reduced:
- `feature_8`
- `feature_9`
- `feature_37`
- `feature_51`

### Constant or Nearly Constant Feature (Drop)
These features have no or very little variance and should be dropped:
- `feature_79`


In [ ]:
# Boxplot for numerical vs target
for col in num_col:
    plt.figure()
    sns.boxplot(x='Bad_label', y=col, data=data)
    plt.title(f"{col} vs Bad_label")
    plt.show()

# Countplot for categorical vs target
for col in cat_col:
    plt.figure()
    sns.countplot(x=col, hue='Bad_label', data=data)
    plt.title(f"{col} vs Bad_label")
    plt.xticks(rotation=45)
    plt.show()
    

In [ ]:
import math
# Define number of columns per row in the grid
n_cols = 5

# Total number of plots
n_plots = len(num_col)

# Calculate rows needed
n_rows = math.ceil(n_plots / n_cols)

# Set figure size based on number of rows/columns
plt.figure(figsize=(n_cols * 4, n_rows * 3))

# Loop through and create subplots
for idx, col in enumerate(num_col):
    plt.subplot(n_rows, n_cols, idx + 1)
    sns.boxplot(y=data[col], color='skyblue')
    plt.title(col, fontsize=9)
    plt.tight_layout()

plt.suptitle("Boxplots for Outlier Detection", fontsize=16, y=1.02)
plt.show()

In [ ]:
# Define a list of columns that are suspected or known to contain outliers in the 'data' DataFrame
# These columns may have extreme values or deviations that could affect analysis or modeling
outliers_col =  [ 
    "acct_type","high_credit_amt","cur_balance_amt","amt_past_due","creditlimit","cashlimit","enq_purpose","enq_amt",
    "feature_3","feature_26","feature_56","feature_66","feature_71"]

In [ ]:
# Loop through each column in the 'outliers_col' list to handle outliers
for i in outliers_col:
    
    # Calculate the first quartile (Q1) of the column 'i' using the quantile function
    Q1 = data[i].quantile(0.25)
    
    # Calculate the third quartile (Q3) of the column 'i'
    Q3 = data[i].quantile(0.75)
    
    # Calculate the Interquartile Range (IQR) which is the difference between Q3 and Q1
    IQR = Q3 - Q1
    
    # Define the lower bound for outliers as Q1 - 1.5 * IQR
    lower_bound = Q1 - 1.5 * IQR
    
    # Define the upper bound for outliers as Q3 + 1.5 * IQR
    upper_bound = Q3 + 1.5 * IQR
    
    # Replace values in the column 'i' that are below the lower bound or above the upper bound with the median of the column
    # This helps to handle outliers by replacing them with a value closer to the center of the data
    data.loc[(data[i] > upper_bound) | (data[i] < lower_bound), i] = data[i].median()

In [ ]:
# Set the figure size for the entire plot (20x25) to ensure ample space for multiple subplots
plt.figure(figsize=(20, 25))

# Initialize a counter variable 'num' to manage the subplot positions
num = 1

# Loop through each numeric column in the 'num_col' list
for i in num_col:
    
    # Ensure the loop runs only if 'num' is less than or equal to the total number of numeric columns
    if num <= len(num_col):
        
        # Create a subplot with a 9x9 grid layout, where 'num' determines the position of each plot
        plt.subplot(9, 9, num)
        
        # Plot a boxplot for the current numeric column 'i'
        # Boxplots help visualize the distribution and identify outliers for each numeric feature
        sns.boxplot(x=data[i]) 
        
        # Set the x-axis label for each subplot to the current column name
        plt.xlabel(i, fontsize=20)
        
        # Increment the 'num' variable to move to the next subplot position
        num += 1 

# Adjust the layout to ensure the subplots don't overlap and are spaced neatly
plt.tight_layout()

# Display all the generated subplots (boxplots)
plt.show()

In [ ]:
# Use pandas' 'get_dummies' function to convert categorical column 'feature_1' into dummy/indicator variables
# This is a common step for one-hot encoding categorical variables, where each unique category in 'feature_1' gets its own column
# The 'dtype=int' ensures the dummy variables are of integer type (0 or 1) instead of the default float type.
data = pd.get_dummies(data, columns=['feature_1'], dtype=int)


In [ ]:
# Convert the 'opened_dt' column to datetime format using pandas' 'to_datetime' function
# This ensures the column is treated as a datetime object, allowing for time-based operations
data['opened_dt'] = pd.to_datetime(data['opened_dt'])

# Extract the year, month, and day from the 'opened_dt' datetime column and create new columns
# 'data['year']' will store the year part of the 'opened_dt'
# 'data['month']' will store the month part of the 'opened_dt'
# 'data['day']' will store the day part of the 'opened_dt'
data['year'], data['month'], data['day'] = data['opened_dt'].dt.year, data['opened_dt'].dt.month, data['opened_dt'].dt.day


In [ ]:
# Fill missing values (NaNs) in the 'year' column with the most frequent value (mode) of that column
# The 'data['year'].mode()[0]' returns the most frequent value (the mode) of the 'year' column
# 'inplace=True' ensures that the changes are applied directly to the 'year' column in the 'data' DataFrame without creating a new variable
data['year'].fillna(data['year'].mode()[0], inplace=True)


In [ ]:
# Fill missing values (NaNs) in the 'month' column with the most frequent value (mode) of that column
# The 'data['month'].mode()[0]' returns the most frequent value (the mode) of the 'month' column
# 'inplace=True' ensures that the changes are applied directly to the 'month' column in the 'data' DataFrame without creating a new variable
data['month'].fillna(data['month'].mode()[0], inplace=True)

In [ ]:
# Fill missing values (NaNs) in the 'month' column with the most frequent value (mode) of that column
# The 'data['month'].mode()[0]' returns the most frequent value (the mode) of the 'month' column
# 'inplace=True' ensures that the changes are applied directly to the 'month' column in the 'data' DataFrame without creating a new variable
data['month'].fillna(data['month'].mode()[0], inplace=True)

In [ ]:
# Fill missing values (NaNs) in the 'day' column with the most frequent value (mode) of that column
# The 'data['day'].mode()[0]' returns the most frequent value (the mode) of the 'day' column
# 'inplace=True' ensures that the changes are applied directly to the 'day' column in the 'data' DataFrame without creating a new variable
data['day'].fillna(data['day'].mode()[0], inplace=True)

In [ ]:
# Convert the 'closed_dt' column to datetime format using pandas' 'to_datetime' function
# This ensures the column is treated as a datetime object, allowing for time-based operations
data['closed_dt'] = pd.to_datetime(data['closed_dt'])

# Extract the year, month, and day from the 'closed_dt' datetime column and create new columns
# 'data['year']' will store the year part of the 'closed_dt'
# 'data['month']' will store the month part of the 'closed_dt'
# 'data['day']' will store the day part of the 'closed_dt'
data['year_close'], data['month_close'], data['day_close'] = data['closed_dt'].dt.year, data['closed_dt'].dt.month, data['closed_dt'].dt.day

In [ ]:
# Fill missing values (NaNs) in the 'year_close' column with the most frequent value (mode) of that column
# The 'data['year_close'].mode()[0]' returns the most frequent value (the mode) of the 'year_close' column
# 'inplace=True' ensures that the changes are applied directly to the 'year_close' column in the 'data' DataFrame without creating a new variable
data['year_close'].fillna(data['year_close'].mode()[0], inplace=True)

In [ ]:
# Fill missing values (NaNs) in the 'month_close' column with the most frequent value (mode) of that column
# The 'data['month_close'].mode()[0]' returns the most frequent value (the mode) of the 'month_close' column
# 'inplace=True' ensures that the changes are applied directly to the 'month_close' column in the 'data' DataFrame without creating a new variable
data['month_close'].fillna(data['month_close'].mode()[0], inplace=True)

In [ ]:
# Drop the 'opened_dt' and 'closed_dt' columns from the DataFrame as they are no longer needed for analysis
# The 'axis=1' argument specifies that we are dropping columns (as opposed to rows)
# 'inplace=True' ensures that the changes are applied directly to the 'data' DataFrame without creating a new one
data.drop(['opened_dt', 'closed_dt'], axis=1, inplace=True)


In [ ]:
# Select columns from the DataFrame that have data types of 'object' (i.e., categorical or string columns)
# 'data.select_dtypes(include=['object'])' returns all columns that are of type 'object', which typically represents categorical variables
# '.columns.tolist()' extracts the column names from the resulting DataFrame and converts them into a list
# This will give a list of all the categorical columns in the 'data' DataFrame
cat_col = data.select_dtypes(include=['object']).columns.tolist()


In [ ]:
# Select columns from the DataFrame that have data types of 'object' (i.e., categorical or string columns)
# 'data.select_dtypes(include=['object'])' returns all columns that are of type 'object', which typically represents categorical variables
# '.columns.tolist()' extracts the column names from the resulting DataFrame and converts them into a list
# This will give a list of all the categorical columns in the 'data' DataFrame
cat_col = data.select_dtypes(include=['object']).columns.tolist()

In [ ]:
data.shape

In [ ]:
len(cat_col)

In [ ]:
data.isnull().sum()

In [ ]:
# Importing the LabelEncoder class from scikit-learn's preprocessing module
from sklearn.preprocessing import LabelEncoder 

# Initialize a LabelEncoder object (LE) that will be used to encode categorical labels into numeric values
LE = LabelEncoder()

# Apply label encoding to each categorical column in the 'cat_col' list
# The 'fit_transform()' method learns the mapping of each category to a numeric value and then transforms the column to numeric values
for i in cat_col:
    data[i] = LE.fit_transform(data[i])


### Feature Selection :-
* Feature selection is the process of choosing the most relevant variables from a dataset that contribute significantly to a model’s predictive power. By reducing the number of features, it simplifies the model, improves training efficiency, and can enhance accuracy by removing noise or irrelevant data. Common techniques include correlation analysis, mutual information, and advanced methods like recursive feature elimination (RFE) and Lasso regularization.

In [ ]:
# Select all columns with numerical data types (int and float) from the DataFrame using 'np.number'
num_cols = data.select_dtypes(include=[np.number])

# Compute the correlation matrix for the numerical columns using the '.corr()' method
# The '.corr()' method calculates pairwise correlation of columns (default is Pearson correlation)
# The '.abs()' function is used to get the absolute values of the correlation coefficients, as we are interested in the magnitude of the correlation, regardless of the sign
correlation_matrix = num_cols.corr().abs()

# Iterate through the upper triangle of the correlation matrix to find pairs of columns with high correlation
# The outer loop iterates through the rows of the correlation matrix
for i in range(len(correlation_matrix.columns)):
    # The inner loop iterates through the columns before the current row (avoiding duplicate checks)
    for j in range(i):
        # Get the correlation value between the pair of columns
        corr_value = correlation_matrix.iloc[i, j]
        # If the absolute correlation value is greater than 0.90, print the pair of columns with high correlation
        if abs(corr_value) > 0.90:
            # Print the pair of columns and their correlation coefficient
            print(f"High correlation: {correlation_matrix.columns[i]} ↔ {correlation_matrix.columns[j]} (r = {corr_value:.2f})")


In [ ]:
data.drop(['feature_9','feature_29','feature_55','feature_59','feature_67','feature_68','feature_69','feature_34','feature_68' ],axis=1,inplace=True)

In [ ]:
data.shape

In [ ]:
data.head()

### Model Creation Process :-

In [ ]:
x = data.drop("Bad_label",axis=1)
y = data.Bad_label

### Scalling Technique:-
* Scaling is very important of continous features because some model give more priproty to that features which has higest values campare to another features. that's why we use Scaling to make all values into certain range.
 
* Basically there are 2 type of scaling.

1. **MinMax Scaling**
2. **Standardization Scaling**

* MinMax Scaling transform all data between 0 to 1  range.
* Standardization Scaling transform all data between -3 to +3 range. it use z-score to tranform data.

In [ ]:
# Importing the StandardScaler class from scikit-learn's preprocessing module
from sklearn.preprocessing import StandardScaler

# Initialize the StandardScaler object, which will be used to standardize the features
# StandardScaler standardizes features by removing the mean and scaling to unit variance (z-score normalization)
scaler = StandardScaler()

In [ ]:
# Standardize the feature set 'x' using the 'scaler' object (StandardScaler)
# 'fit_transform()' first computes the mean and standard deviation of each feature in 'x' (fit),
# and then scales the data by transforming it (subtracting the mean and dividing by the standard deviation)
# The result is stored in 'x_scale', which is the standardized version of the input features 'x'
x_scale = scaler.fit_transform(x)

## PCA:- Principle Components Analysis

* Principal Component Analysis (PCA) is a dimensionality reduction technique that transforms high-dimensional data into a smaller set of uncorrelated variables called principal components. These components capture the most variance in the data, making it possible to reduce complexity while retaining important information. PCA is useful for visualizing data in lower dimensions and improving model performance by removing noise and redundancy.

In [ ]:
# Importing the PCA (Principal Component Analysis) class from scikit-learn's decomposition module
from sklearn.decomposition import PCA

# Initialize the PCA object, specifying the number of components to keep after dimensionality reduction
# 'n_components=15' means that PCA will reduce the data to 15 principal components, 
# which are the new features that capture the most variance in the data
pca = PCA(n_components=15)

In [ ]:
# Apply PCA (Principal Component Analysis) to the scaled feature set 'x_scale'
# The 'fit_transform()' method first computes the principal components (fit) and then transforms the data (x_scale) 
# into a lower-dimensional space using the top 15 principal components as specified earlier.
# The result is stored in 'data_pca', which is the transformed dataset with reduced dimensions (15 components).
data_pca = pca.fit_transform(x_scale)

In [ ]:
data_pca

In [ ]:
pca.explained_variance_

In [ ]:
from sklearn.model_selection import train_test_split
x_train,x_test,y_train,y_test = train_test_split(data_pca,y,test_size=0.25,random_state=4)

In [ ]:
x_train

In [ ]:
x_test

In [ ]:
y_train

In [ ]:
y_test

In [ ]:
# Importing various classification models from scikit-learn, CatBoost, and XGBoost
from sklearn.linear_model import LogisticRegression  # Logistic Regression, used for binary or multiclass classification
from sklearn.neighbors import KNeighborsClassifier  # K-Nearest Neighbors, a simple algorithm for classification based on proximity to neighbors
from sklearn.svm import SVC, LinearSVC  # Support Vector Classifier (SVC), used for classification tasks with high-dimensional data; LinearSVC is for linear decision boundaries
from sklearn.tree import DecisionTreeClassifier  # Decision Tree, a non-linear classifier that splits data based on feature values
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier, AdaBoostClassifier  # Ensemble methods for improved performance:
# RandomForest (multiple decision trees), Gradient Boosting (sequential learning), and AdaBoost (boosting weak learners)
from sklearn.neural_network import MLPClassifier  # Multi-layer Perceptron (MLP), a type of neural network for classification
from catboost import CatBoostClassifier  # CatBoost, a gradient boosting algorithm designed for categorical data
from xgboost import XGBClassifier  # XGBoost, a highly efficient gradient boosting library for classification and regression tasks

In [ ]:
# Importing classification evaluation metrics: confusion_matrix, accuracy_score, precision_score, 
# classification_report, f1_score, and roc_auc_score to assess model performance across various metrics.
from sklearn.metrics import confusion_matrix,accuracy_score,precision_score,classification_report,f1_score,roc_auc_score 

In [ ]:
# Defining a dictionary of classification models with their respective hyperparameters
models = {
    'Logistic Regression': LogisticRegression(max_iter=1000),  # Logistic Regression with max iterations set to 1000
    'K-Nearest Neighbors': KNeighborsClassifier(),  # K-Nearest Neighbors classifier
    'Support Vector Machine': SVC(),  # Support Vector Machine classifier
    'Linear SVM': LinearSVC(max_iter=10000),  # Linear Support Vector Machine with max iterations set to 10000
    'Decision Tree': DecisionTreeClassifier(),  # Decision Tree classifier
    'Random Forest': RandomForestClassifier(),  # Random Forest classifier (ensemble method)
    'Gradient Boosting': GradientBoostingClassifier(),  # Gradient Boosting classifier (ensemble method)
    'AdaBoost': AdaBoostClassifier(),  # AdaBoost classifier (ensemble method)
    'MLP Classifier': MLPClassifier(max_iter=1000),  # Multi-layer Perceptron classifier with max iterations set to 1000
    'CatBoost': CatBoostClassifier(verbose=0),  # CatBoost classifier with verbosity turned off
    'XGBoost': XGBClassifier(use_label_encoder=False, eval_metric='logloss')  # XGBoost classifier with settings for label encoding and evaluation metric
}

In [ ]:
# Importing evaluation metrics for classification model performance
a = accuracy_score  # Accuracy metric
p = precision_score  # Precision metric
f = f1_score  # F1 score metric

# Function to evaluate model performance
def evaluate_model(true, predicted):
    accuracy_score = a(true, predicted)  # Calculate accuracy
    precision_score = p(true, predicted)  # Calculate precision
    f1_score_score = f(true, predicted)  # Calculate F1 score
    auc = roc_auc_score(true, predicted)  # Calculate ROC AUC score
    gini = 2 * auc - 1  # Calculate Gini coefficient (derived from AUC)
    return accuracy_score, precision_score, f1_score_score, (gini * 100)  # Return evaluation metrics

In [ ]:
# Initialize lists to store model names and their corresponding accuracy scores
model_list = []  # List to store model names
accuracy_score_list = []  # List to store model test accuracy scores

# Step 3: Loop through models and evaluate their performance
for model_name, model in models.items():
    model.fit(x_train, y_train)  # Train the model using the training data

    # Predictions on training and test sets
    y_train_pred = model.predict(x_train)  # Predict on training data
    y_test_pred = model.predict(x_test)  # Predict on test data

    # Evaluate model performance on both training and test sets
    model_train_accuracy_score, model_train_precision_score, model_train_f1_score, model_train_gini = evaluate_model(y_train, y_train_pred)
    model_test_accuracy_score, model_test_precision_score, model_test_f1_score, model_test_gini = evaluate_model(y_test, y_test_pred)

    # Print and store the results for both training and test sets
    print(f"Model: {model_name}")
    model_list.append(model_name)  # Store model name

    print('Model Performance for Training Set:')
    print("- Accuracy: {:.4f}".format(model_train_accuracy_score))
    print("- Precision: {:.4f}".format(model_train_precision_score))
    print("- F1 Score: {:.4f}".format(model_train_f1_score))
    print("- gini Score: {:.4f}".format(model_train_gini))
    print('----------------------------------')

    print('Model Performance for Test Set:')
    print("- Accuracy: {:.4f}".format(model_test_accuracy_score))
    print("- Precision: {:.4f}".format(model_test_precision_score))
    print("- F1 Score: {:.4f}".format(model_test_f1_score))
    print("- gini Score: {:.4f}".format(model_test_gini))

    accuracy_score_list.append(model_test_accuracy_score)  # Store test accuracy score

    print('='*40)  # Separator for readability
    print('\n')


In [ ]:
# Create a DataFrame to store the model names and their corresponding accuracy scores, 
# and sort the results in descending order based on the accuracy score
result = pd.DataFrame(list(zip(model_list, accuracy_score_list)), columns=['Model Name', 'Accuracy_score']).sort_values(by=["Accuracy_score"],ascending=False)


In [ ]:
result

In [ ]:
result.Accuracy_score = result.Accuracy_score*100

In [ ]:
result

In [ ]:
plt.figure(figsize=(10, 6)) # plotting using matplotlib
num = 0
for i in result :
    if num <= 7:
        a = sns.barplot(x='Model Name', y='Accuracy_score', data=result,palette='viridis')
        plt.title('R² Score Comparison of Models') # plotting using matplotlib
        plt.xlabel('Model Name') # plotting using matplotlib
        plt.ylabel('Accuracy score') # plotting using matplotlib
        # plt.ylim(0, 1.1) # plotting using matplotlib
        plt.xticks(rotation=90) # plotting using matplotlib
        
        for index,value in enumerate(result['Accuracy_score']):
            a.text(index,value + 0.02 , f'{value:.2f}',ha='center')

        num += 1

plt.tight_layout() # plotting using matplotlib
plt.show() # plotting using matplotlib

## Conclusion

### Top Performers:
- **Random Forest**: 96.31%
- **Logistic Regression**, **Support Vector Machine**, **Linear SVM**, **AdaBoost**, and **CatBoost** — all scoring **96.30%**.
- These models show almost identical performance based on the R² score.
- **K Nearest Neighbors**: 96.25%
- **XGBoost**: 96.25%
- **Gradient Boosting**: 96.20%
- **MLP Classifier (Neural Network)**: 95.58%

- **Decision Tree**: 91.13% (significantly lower than the others)

### Observations:
- Most models are performing very similarly, clustering around **96% accuracy**.
- The **Decision Tree** stands out as the weaker model in this comparison.
- Ensemble methods like **Random Forest**, **AdaBoost**, **CatBoost**, **XGBoost**, and **Gradient Boosting** mostly perform well and consistently.


# Report on challange faced :-

### 1. Data Quality Issues
- **Missing Values**: Several fields like `paymenthistory2`, `feature_8`, `feature_9`, etc. have high missingness.
- **Mixed Data Types**: Columns such as `opened_dt`, `last_paymt_dt`, `rateofinterest` contain mixed types (string + numeric).
- **Inconsistent Formats**: Date fields (`opened_dt`, `enquiry_dt`, `closed_dt`) are not standardized and need transformation.

### 2. Data Redundancy & Irrelevance
- Presence of **ID-like variables** (`customer_no`, `upload_dt`) which do not contribute to prediction.
- **Highly correlated features** (e.g., `cur_balance_amt` vs. `high_credit_amt`) create redundancy.
- Some features may be **constant or near-constant**, adding no predictive value.

### 3. Class Imbalance
- Target variable `Bad_label` likely has **fewer bad customers (1)** compared to good ones (0).
- Imbalanced datasets lead to biased models that predict mostly the majority class.

### 4. Privacy & Feature Obfuscation
- Demographic variables (`feature_1` to `feature_79`) are anonymized.
- Lack of semantic meaning makes interpretation and business validation difficult.

### 5. Data Integration Challenges
- Data comes from **multiple sources** (Accounts, Enquiries, Demographics).
- Requires **joining/merging on customer_no**; mismatches or missing keys may cause data loss.

### 6. Outliers & Skewness
- Financial features (`cur_balance_amt`, `enq_amt`, `high_credit_amt`) show **large ranges and skewed distributions**.
- Outliers may distort model performance if not handled.

### 7. Temporal Issues
- Payment history and enquiry data are **time-dependent**.
- Model may overfit if recency trends are not captured properly.

## Model Save :-

In [ ]:
import pickle                             #Importing the pickle module
with open('best_model.pkl', 'wb') as file:
    pickle.dump(RandomForestClassifier(), file)                      #save it to the file

In [ ]:
with open("best_model.pkl","rb") as f:   #Open the file "best_model.pkl" in read-binary mode
  model=pickle.load(f)

In [ ]:
model

                                                      ## END ##